In [1]:
#specify GPU to use
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
from sklearn.model_selection import train_test_split
import openml
import category_encoders as ce
import numpy as np
import sklearn

# Load SpeedDating dataset
dataset = openml.datasets.get_dataset(40536)
X, y, categorical_indicator, attribute_names = dataset.get_data(target=dataset.default_target_attribute)

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)


print("Training set size:", len(X_train))
print("Validation set size:", len(X_valid))
print("Test set size:", len(X_test))


Training set size: 5361
Validation set size: 1341
Test set size: 1676


/home/smarton/anaconda3/envs/GradientBasedTreeLearning/lib/python3.11/site-packages/openml/datasets/functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


In [3]:
X_train.head()

,has_null,wave,gender,age,age_o,d_age,d_d_age,race,race_o,samerace,...,expected_num_interested_in_me,expected_num_matches,d_expected_happy_with_sd_people,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met
813,0,3,female,23.0,21.0,2,[2-3],European/Caucasian-American,European/Caucasian-American,1,...,15.0,3.0,[0-4],[10-20],[3-5],5.0,10.0,[0-5],[7-10],0.0
1677,1,5,female,20.0,18.0,2,[2-3],Black/African American,Asian/Pacific Islander/Asian-American,0,...,NaN,3.0,[5-6],[0-3],[3-5],7.0,7.0,[6-8],[7-10],0.0
2915,1,9,female,28.0,27.0,1,[0-1],Black/African American,Asian/Pacific Islander/Asian-American,0,...,NaN,0.0,[7-10],[0-3],[0-2],NaN,NaN,[0-5],[0-4],NaN
5972,1,15,female,28.0,24.0,4,[4-6],European/Caucasian-American,European/Caucasian-American,1,...,NaN,1.0,[0-4],[0-3],[0-2],3.0,5.0,[0-5],[5-6],0.0
5443,1,14,male,23.0,24.0,1,[0-1],Asian/Pacific Islander/Asian-American,Other,0,...,NaN,4.0,[7-10],[0-3],[3-5],5.0,4.0,[0-5],[0-4],0.0


In [4]:
X_train_raw = X_train.copy()
X_valid_raw = X_valid.copy()
X_test_raw = X_test.copy()

low_cardinality_indices = []
high_cardinality_indices = []

categorical_feature_indices = []
for column_index in range(X_train.shape[1]):
    if categorical_indicator[column_index]:
        categorical_feature_indices.append(column_index)
        if len(X_train.iloc[:,column_index].unique()) < 10:
            low_cardinality_indices.append(X_train.columns[column_index])
        else:
            high_cardinality_indices.append(X_train.columns[column_index])

y_train = y_train.values.codes.astype(np.float64)
y_valid = y_valid.values.codes.astype(np.float64)
y_test = y_test.values.codes.astype(np.float64)

In [5]:
encoder = ce.OrdinalEncoder(cols=X_train.columns[categorical_feature_indices])
encoder.fit(X_train)
X_train = encoder.transform(X_train).astype(np.float64)
X_valid = encoder.transform(X_valid).astype(np.float64)
X_test = encoder.transform(X_test).astype(np.float64)

In [6]:
from GRANDE import GRANDE

params = {
        'depth': 5,
        'n_estimators': 2048,

        'learning_rate_weights': 0.005,
        'learning_rate_index': 0.01,
        'learning_rate_values': 0.01,
        'learning_rate_leaf': 0.01,

        'optimizer': 'SWA',
        'cosine_decay_steps': 0,

        'initializer': 'RandomNormal',

        'loss': 'crossentropy',
        'focal_loss': False,
        'temperature': 0.0,

        'from_logits': True,
        'apply_class_balancing': True,

        'dropout': 0.0,

        'selected_variables': 0.8,
        'data_subset_fraction': 1.0,
}

args = {
    'epochs': 1_000,
    'early_stopping_epochs': 25,
    'batch_size': 64,

    'cat_idx': categorical_feature_indices,
    'objective': 'binary',
    
    'metrics': ['F1'], # F1, Accuracy, R2
    'random_seed': 42,
    'verbose': 1,       
}

model_grande = GRANDE(params=params, args=args)

model_grande.fit(X_train=X_train,
          y_train=y_train,
          X_val=X_valid,
          y_val=y_valid)

preds_grande = model_grande.predict(X_test)

2023-11-02 15:18:16.328503: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-02 15:18:18.344896: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/smarton/anaconda3/envs/GradientBasedTreeLearning/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
202

Epoch 1/1000


2023-11-02 15:18:26.662030: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x22a729d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-02 15:18:26.662089: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2023-11-02 15:18:26.678964: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-02 15:18:27.862780: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8801
2023-11-02 15:18:29.117989: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


84/84 [==============================] - 14s 73ms/step - loss: 0.6070 - val_loss: 0.5224
Epoch 2/1000
84/84 [==============================] - 3s 40ms/step - loss: 0.4613 - val_loss: 0.4119
Epoch 3/1000
84/84 [==============================] - 3s 39ms/step - loss: 0.3799 - val_loss: 0.3676
Epoch 4/1000
84/84 [==============================] - 3s 40ms/step - loss: 0.3457 - val_loss: 0.3522
Epoch 5/1000
84/84 [==============================] - 3s 41ms/step - loss: 0.3321 - val_loss: 0.3478
Epoch 6/1000
84/84 [==============================] - 3s 40ms/step - loss: 0.3243 - val_loss: 0.3464
Epoch 7/1000
84/84 [==============================] - 3s 41ms/step - loss: 0.3194 - val_loss: 0.3452
Epoch 8/1000
84/84 [==============================] - 4s 42ms/step - loss: 0.3153 - val_loss: 0.3432
Epoch 9/1000
84/84 [==============================] - 4s 42ms/step - loss: 0.3117 - val_loss: 0.3419
Epoch 10/1000
84/84 [==============================] - 4s 42ms/step - loss: 0.3086 - val_loss: 0.3411
E

In [7]:
def calculate_sample_weights(y_data):
    class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight = 'balanced', classes = np.unique(y_data), y = y_data)
    #class_weights = class_weights/sum(class_weights)#    
    sample_weights = sklearn.utils.class_weight.compute_sample_weight(class_weight = 'balanced', y =y_data)
    #sample_weights = sample_weights/sum(class_weights)
    return sample_weights

In [8]:
median = X_train.median(axis=0)
X_train= X_train.fillna(median)
X_vali = X_valid.fillna(median)
X_test = X_test.fillna(median)

encoder = ce.LeaveOneOutEncoder(cols=high_cardinality_indices)
encoder.fit(X_train, y_train)
X_train = encoder.transform(X_train)
X_valid = encoder.transform(X_valid)
X_test = encoder.transform(X_test)

encoder = ce.OneHotEncoder(cols=low_cardinality_indices)
encoder.fit(X_train)
X_train = encoder.transform(X_train)
X_valid = encoder.transform(X_valid)
X_test = encoder.transform(X_test)

median = X_train.median(axis=0)
X_train = X_train.fillna(median)
X_valid = X_valid.fillna(median)
X_test = X_test.fillna(median)

quantile_noise = 1e-4
quantile_train = np.copy(X_train.values).astype(np.float64)
np.random.seed(42)
stds = np.std(quantile_train, axis=0, keepdims=True)
noise_std = quantile_noise / np.maximum(stds, quantile_noise)
quantile_train += noise_std * np.random.randn(*quantile_train.shape)       

scaler = sklearn.preprocessing.QuantileTransformer(output_distribution='normal')
scaler.fit(quantile_train)

X_train = scaler.transform(X_train.values.astype(np.float64))
X_valid = scaler.transform(X_valid.values.astype(np.float64))
X_test = scaler.transform(X_test.values.astype(np.float64))

In [9]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(n_estimators=1000, early_stopping_rounds=20)
model_xgb.fit(X_train, 
                y_train, 
                sample_weight=calculate_sample_weights(y_train), 
                eval_set=[(X_valid, y_valid)], 
                sample_weight_eval_set=[calculate_sample_weights(y_valid)]
             )

preds_xgb = model_xgb.predict_proba(X_test)

[0]	validation_0-logloss:0.61617
[1]	validation_0-logloss:0.58583
[2]	validation_0-logloss:0.57171
[3]	validation_0-logloss:0.56602
[4]	validation_0-logloss:0.57168
[5]	validation_0-logloss:0.56915
[6]	validation_0-logloss:0.57535
[7]	validation_0-logloss:0.58421
[8]	validation_0-logloss:0.59058
[9]	validation_0-logloss:0.59599
[10]	validation_0-logloss:0.60741
[11]	validation_0-logloss:0.62176
[12]	validation_0-logloss:0.62515
[13]	validation_0-logloss:0.63687
[14]	validation_0-logloss:0.64298
[15]	validation_0-logloss:0.64503
[16]	validation_0-logloss:0.64726
[17]	validation_0-logloss:0.65866
[18]	validation_0-logloss:0.66732
[19]	validation_0-logloss:0.66513
[20]	validation_0-logloss:0.67006
[21]	validation_0-logloss:0.67947
[22]	validation_0-logloss:0.68068


In [10]:
encoder = ce.OrdinalEncoder(cols=X_train_raw.columns[categorical_feature_indices])
encoder.fit(X_train_raw)
X_train_raw = encoder.transform(X_train_raw)
X_valid_raw = encoder.transform(X_valid_raw)
X_test_raw = encoder.transform(X_test_raw)

median = X_train_raw.median(axis=0)
X_train_raw = X_train_raw.fillna(median)
X_valid_raw = X_valid_raw.fillna(median)
X_test_raw = X_test_raw.fillna(median)

In [11]:
from catboost import CatBoostClassifier, Pool

model_catboost = CatBoostClassifier(n_estimators=1000, 
                                    early_stopping_rounds=20)
train_data = Pool(
        data=X_train_raw,
        label=y_train,
        cat_features=categorical_feature_indices,
        weight=calculate_sample_weights(y_train)
    )

eval_data = Pool(
        data=X_valid_raw,
        label=y_valid,
        cat_features=categorical_feature_indices,
        weight=calculate_sample_weights(y_valid),
    )

model_catboost.fit(X=train_data, 
                   eval_set=eval_data)

preds_catboost = model_catboost.predict_proba(X_test_raw)

Learning rate set to 0.047977
0:	learn: 0.6738749	test: 0.6755616	best: 0.6755616 (0)	total: 180ms	remaining: 2m 59s
1:	learn: 0.6590376	test: 0.6621054	best: 0.6621054 (1)	total: 250ms	remaining: 2m 4s
2:	learn: 0.6476492	test: 0.6530350	best: 0.6530350 (2)	total: 311ms	remaining: 1m 43s
3:	learn: 0.6334984	test: 0.6409063	best: 0.6409063 (3)	total: 420ms	remaining: 1m 44s
4:	learn: 0.6211706	test: 0.6295918	best: 0.6295918 (4)	total: 516ms	remaining: 1m 42s
5:	learn: 0.6089204	test: 0.6190270	best: 0.6190270 (5)	total: 611ms	remaining: 1m 41s
6:	learn: 0.5978930	test: 0.6102024	best: 0.6102024 (6)	total: 707ms	remaining: 1m 40s
7:	learn: 0.5884188	test: 0.6025458	best: 0.6025458 (7)	total: 798ms	remaining: 1m 38s
8:	learn: 0.5827542	test: 0.5988541	best: 0.5988541 (8)	total: 836ms	remaining: 1m 32s
9:	learn: 0.5709456	test: 0.5900052	best: 0.5900052 (9)	total: 928ms	remaining: 1m 31s
10:	learn: 0.5629534	test: 0.5835565	best: 0.5835565 (10)	total: 1.02s	remaining: 1m 32s
11:	learn: 0

In [12]:
if args['objective'] == 'binary':
    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_grande[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_grande[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_grande[:,1], average='macro', multi_class='ovo')

    print('Accuracy GRANDE:', accuracy)
    print('F1 Score GRANDE:', f1_score)
    print('ROC AUC GRANDE:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_xgb[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_xgb[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_xgb[:,1], average='macro', multi_class='ovo')

    print('Accuracy XGB:', accuracy)
    print('F1 Score XGB:', f1_score)
    print('ROC AUC XGB:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_catboost[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_catboost[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_catboost[:,1], average='macro', multi_class='ovo')

    print('Accuracy CatBoost:', accuracy)
    print('F1 Score CatBoost:', f1_score)
    print('ROC AUC CatBoost:', roc_auc)
    print('\n')
elif args['objective'] == 'classification':
    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_grande, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_grande, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_grande, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy GRANDE:', accuracy)
    print('F1 Score GRANDE:', f1_score)
    print('ROC AUC GRANDE:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_xgb, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_xgb, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_xgb, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy XGB:', accuracy)
    print('F1 Score XGB:', f1_score)
    print('ROC AUC XGB:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_catboost, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_catboost, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_catboost, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy CatBoost:', accuracy)
    print('F1 Score CatBoost:', f1_score)
    print('ROC AUC CatBoost:', roc_auc)
    print('\n')
else:
    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, np.round(preds_grande[:,1]))
    r2_score = sklearn.metrics.r2_score(y_test, np.round(preds_grande[:,1]), average='macro')

    print('MAE GRANDE:', mean_absolute_error)
    print('R2 Score GRANDE:', r2_score)
    print('\n')

    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, np.round(preds_xgb[:,1]))
    r2_score = sklearn.metrics.r2_score(y_test, np.round(preds_xgb[:,1]), average='macro')

    print('MAE XGB:', mean_absolute_error)
    print('R2 Score XGB:', r2_score)
    print('\n')

    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, np.round(preds_catboost[:,1]))
    r2_score = sklearn.metrics.r2_score(y_test, np.round(preds_catboost[:,1]), average='macro')

    print('MAE CatBoost:', mean_absolute_error)
    print('R2 Score CatBoost:', r2_score)
    print('\n')

Accuracy GRANDE: 0.8520286396181385
F1 Score GRANDE: 0.7104289281255574
ROC AUC GRANDE: 0.8663591233258829


Accuracy XGB: 0.7673031026252983
F1 Score XGB: 0.6787340458159341
ROC AUC XGB: 0.8000830316849913


Accuracy CatBoost: 0.7869928400954654
F1 Score CatBoost: 0.7046369469890426
ROC AUC CatBoost: 0.8625913411247657


